In [154]:
import pandas as pd
from pandas_ta import bbands, rsi, vwap
from Datamanager.DGMT import DGMT

In [155]:
dmgt = DGMT('../BTCUSDT.csv','time')
dmgt.change_resolution('10min')
df = dmgt.df



df['rsi'] = rsi(df.close, 14)
df['vwap'] = vwap(df.high, df.low, df.close, df.volume)
bbands = bbands(df.close, 14, 2.0)
df = df.join(bbands)
df.drop(columns=['BBM_14_2.0', 'BBB_14_2.0', 'BBP_14_2.0'], inplace=True)
df.dropna(inplace=True)
df.rename(columns = {'BBL_14_2.0':'bbl', 'BBU_14_2.0':'bbu'}, inplace=True)



In [156]:
df

,volume,open,low,high,close,t_plus,rsi,vwap,bbl,bbu,atr
time,,,,,,,,,,,
2017-12-01 02:20:00,32.961861,9590.01,9510.00,9607.90,9535.04,9536.03,31.894021,9692.309440,9478.865538,9874.235891,104.551319
2017-12-01 02:30:00,43.082735,9536.03,9525.00,9646.78,9646.78,9644.74,43.364134,9686.923997,9494.821163,9826.185980,106.385399
2017-12-01 02:40:00,26.173022,9644.74,9585.00,9649.98,9590.12,9649.97,39.711922,9684.048929,9495.821505,9795.202781,102.126752
2017-12-01 02:50:00,47.488233,9649.97,9627.08,9720.03,9719.00,9702.00,50.022705,9684.338754,9525.548755,9747.554102,104.897267
2017-12-01 03:00:00,59.269479,9702.00,9673.99,9718.99,9678.03,9678.03,47.255883,9684.771280,9527.041688,9742.909740,99.089674
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 23:10:00,66.296690,30462.00,30461.16,30480.00,30480.00,30480.00,54.211079,30480.122147,30398.798352,30542.780220,59.219642
2023-06-30 23:20:00,168.454310,30480.00,30479.99,30513.39,30508.74,30508.75,57.669376,30480.161123,30417.704602,30539.976826,57.375382
2023-06-30 23:30:00,185.367950,30508.75,30477.04,30544.44,30483.98,30483.97,53.892939,30480.206158,30434.407270,30533.272730,58.091426


In [157]:
def vwap_signal(backcandles, length, df):
    vwap_signal = [0] * len(df)
    for row in range(backcandles , length):
        upt = 1
        dnt = 1
        for i in range(row - backcandles, row+1):
            if max(df.open[i], df.close[i]) >= df.vwap[i]:
                dnt = 0
            if min(df.open[i], df.close[i]) <= df.vwap[i]:
                upt = 0


        if upt == 1 and dnt == 1:
            vwap_signal[row] = 0
        elif upt == 1 and dnt == 0:
            vwap_signal[row] = 1
        elif upt == 0 and dnt == 1:
            vwap_signal[row] = -1
        # else:
        #     vwap_signal[row] = 0

    return vwap_signal



vsig = vwap_signal(15, len(df), df)
df['vwap_sig'] = vsig

In [158]:
def bband_sig(df):
    bbandsig = [0] * len(df)
    n = 0
    for index, row in df.iterrows():

        if (row['close'] < row['bbl']):
            bbandsig[n] = 1
        elif (row['close'] > row['bbu']):
            bbandsig[n] = -1
        else: 
            bbandsig[n] = 0
        n+=1
    return bbandsig

bbandsig = bband_sig(df)
df['bbandsig'] = bbandsig

In [159]:
def generate_signal(df):
    df['long'] = ((df.vwap_sig == 1) & (df.bbandsig == 1) & (df.rsi < 45)) * 1
    df['short'] = ((df.vwap_sig == -1) & (df.bbandsig == -1) & (df.rsi > 55)) * -1
    df['entry'] = df['long'] + df['short']

generate_signal(df)


In [161]:
(df['entry'] == 1).sum()

1204